## 環境
Pytourchの環境設定、Pythonライブラリの設定などを行う。Google Colabデフォルトのpytorchのバージョンを確認して、違ったら念のため高田さんのJetson nano環境へ合わせる。

この処理不要かも。学習データをtrt形式に変更後、うまく動けば環境を合わせる必要なし。後で検証する。

In [1]:
!python3 -c 'import torch; print(torch.__version__) '
!python3 -c 'import torchvision; print(torchvision.__version__) '

1.7.0+cu101
0.8.1+cu101


In [2]:
!pip uninstall torch
!pip uninstall torchvision
!pip install torch===1.4.0 torchvision===0.5.0

Uninstalling torch-1.7.0+cu101:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.6/dist-packages/caffe2/*
    /usr/local/lib/python3.6/dist-packages/torch-1.7.0+cu101.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.7.0+cu101
Uninstalling torchvision-0.8.1+cu101:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/torchvision-0.8.1+cu101.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torchvision.libs/libcudart.c740f4ef.so.10.1
    /usr/local/lib/python3.6/dist-packages/torchvision.libs/libjpeg.ceea7512.so.62
    /usr/local/lib/python3.6/dist-packages/torchvision.libs/libpng16.7f72a3c5.so.16
    /usr/local/lib/python3.6/dist-packages/torchvision.libs/libz.1328edc3.so.1
    /usr/local/lib/python3.6/dist-packages/torchvision/*
Proceed (y/n)? y
  Successfully uninstalled torchvision-0.8.1+cu101
     |████████████████████████████████|

versionの確認およびGPU使用可能か確認する。TureならGPU使用可能。

In [3]:
!python3 -c 'import torch; print(torch.__version__) '
!python3 -c 'import torchvision; print(torchvision.__version__) '
import torch
print(torch.cuda.is_available())
torch.cuda.current_device()

1.4.0
0.5.0
True


0

## Google Driveマウント


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


作業ディレクトリに移動する。

In [5]:
cd drive/My Drive/Colab Notebooks/AI_Race

/content/drive/My Drive/Colab Notebooks/AI_Race


## Gitクローン
~~必要に応じてgitから環境をクローンする。まずは初回だけ。~~
ai_race/ai_race/learning/scripts直下のファイルがあればOKだと思われる。以下はseigotからフォークしたものをすべてGoogleDrive上にクローンしてしまった例。


In [24]:
# !git clone https://github.com/iwatadive28/ai_race.git

#Train実行
スクリプト直下まで移動してtrain.pyを実行してみる。

In [6]:
cd ai_race/ai_race/learning/scripts

/content/drive/My Drive/Colab Notebooks/AI_Race/ai_race/ai_race/learning/scripts


## Train実行
作業ディレクトリ直下まで移動。ここではtrain.pyと同等の処理を小分けにして行う。

### モジュールのインポート

In [7]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import os
import io
import argparse
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from MyDataSet import MyDataset

### train関数

In [8]:
def train(model, device, train_loader, criterion, optimizer):
	model.train()

	output_list = []
	target_list = []
	running_loss = 0.0
	for batch_idx, (inputs, targets) in enumerate(train_loader):
		# Forward processing.
		inputs, targets = inputs.to(device), targets.to(device)
		outputs = model(inputs)
		loss = criterion(outputs, targets)

		# Backward processing.
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		# Set data to calculate score.
		output_list += [int(o.argmax()) for o in outputs]
		target_list += [int(t) for t in targets]
		running_loss += loss.item()

		# Calculate score at present.
		train_acc, train_loss = calc_score(output_list, target_list, running_loss, train_loader)
		if batch_idx % 10 == 0 and batch_idx != 0:
			stdout_temp = 'batch: {:>3}/{:<3}, train acc: {:<8}, train loss: {:<8}'
			print(stdout_temp.format(batch_idx, len(train_loader), train_acc, train_loss))

	# Calculate score.
	train_acc, train_loss = calc_score(output_list, target_list, running_loss, train_loader)

	return train_acc, train_loss

### test関数

In [9]:
def test(model, device, test_loader, criterion):
	model.eval()

	output_list = []
	target_list = []
	running_loss = 0.0
	for batch_idx, (inputs, targets) in enumerate(test_loader):
		# Forward processing.
		inputs, targets = inputs.to(device), targets.to(device)
		outputs = model(inputs)
		loss = criterion(outputs, targets)
		
		# Set data to calculate score.
		output_list += [int(o.argmax()) for o in outputs]
		target_list += [int(t) for t in targets]
		running_loss += loss.item()
		
	test_acc, test_loss = calc_score(output_list, target_list, running_loss, test_loader)

	return test_acc, test_loss

In [10]:
def calc_score(output_list, target_list, running_loss, data_loader):
	# Calculate accuracy.
	#result = classification_report(output_list, target_list) #, output_dict=True)
	#acc = round(result['weighted avg']['f1-score'], 6)
	acc = round(f1_score(output_list, target_list, average='micro'), 6)
	loss = round(running_loss / len(data_loader.dataset), 6)

	return acc, loss

### 初期値設定
train.pyでは引数で設定している個所をべた書きで指定する

In [19]:
dataset_name         = 'sim_race'
data_csv             = '../../../../DataSet/_2021-01-04-13-37-29/_2021-01-04-13-37-29.csv'
model_name           = 'keyboad_test_ResNet18'
model_ckpt_dir       = './model_0104_2/checkpoints/'
model_ckpt_path_temp = '/work/experiments/models/checkpoints/{}_{}_epoch={}.pth'
n_epoch              = 20
lr                   = 0.1
test_interval        = 2
save_model_interval  = 2


# Make directory.
os.makedirs(model_ckpt_dir, exist_ok=True)

print(data_csv)
# Validate paths.
assert os.path.exists(data_csv)
assert os.path.exists(model_ckpt_dir)


../../../../DataSet/_2021-01-04-13-37-29/_2021-01-04-13-37-29.csv


### main関数
Deviceの設定、データロード、モデル設定、オプティマイザーの設定などステップ的に実行する。
元のtrain.pyから引数部のみ変更した。


In [22]:
# Set device.
print('Setting Device ...')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ROOT_DIR = ""
imgDataset = MyDataset(data_csv, ROOT_DIR, transform=transforms.ToTensor())
print('done!')

# Load dataset.
print('Loading dataset...')
train_data, test_data = train_test_split(imgDataset, test_size=0.2)
pd.to_pickle(test_data, "test_data.pkl")
del test_data
train_loader = torch.utils.data.DataLoader(train_data, batch_size=20, shuffle=True)
print('done!')

print('Setting model...')
# Set a model.
model = models.resnet18()
model.train()
model.fc = torch.nn.Linear(512, 3)
model = model.to(device)
print('done!')

print('Setting optimizer...')
# Set loss function and optimization function.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
print('done!')


Setting Device ...
done!
Loading dataset...
done!
Setting model...
done!
Setting optimizer...
done!


### train


In [23]:
# Train and test.
print('Training...')
for epoch in range(n_epoch):
  # Train and test a model.
  train_acc, train_loss = train(model, device, train_loader, criterion, optimizer)
  
  # Output score.
  if(epoch%test_interval == 0):
    pd.to_pickle(train_data, "train_data.pkl")
    del train_data
    
    test_data   = pd.read_pickle("test_data.pkl")
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=20, shuffle=True)
    del test_data
    test_acc, test_loss = test(model, device, test_loader, criterion)
    del test_loader

    stdout_temp = 'epoch: {:>3}, train acc: {:<8}, train loss: {:<8}, test acc: {:<8}, test loss: {:<8}'
    print(stdout_temp.format(epoch+1, train_acc, train_loss, test_acc, test_loss))

    train_data = pd.read_pickle("train_data.pkl")
  else:	
    stdout_temp = 'epoch: {:>3}, train acc: {:<8}, train loss: {:<8}' #, test acc: {:<8}, test loss: {:<8}'
    print(stdout_temp.format(epoch+1, train_acc, train_loss)) #, test_acc, test_loss))

  # Save a model checkpoint.
  if(epoch%save_model_interval == 0):
    model_ckpt_path_temp = model_ckpt_dir + '{}_{}_epoch={}.pth'
    model_ckpt_path = model_ckpt_path_temp.format(dataset_name, model_name, epoch+1)
    torch.save(model.state_dict(), model_ckpt_path)
    print('Saved a model checkpoint at {}'.format(model_ckpt_path))
    print('')

print('done!')

Training...
batch:  10/24 , train acc: 0.6     , train loss: 0.019484
batch:  20/24 , train acc: 0.62381 , train loss: 0.035225
epoch:   1, train acc: 0.626305, train loss: 0.040336, test acc: 0.416667, test loss: 0.0933  
Saved a model checkpoint at ./model_0104_2/checkpoints/sim_race_keyboad_test_ResNet18_epoch=1.pth

batch:  10/24 , train acc: 0.645455, train loss: 0.017029
batch:  20/24 , train acc: 0.657143, train loss: 0.029981
epoch:   2, train acc: 0.655532, train loss: 0.034731
batch:  10/24 , train acc: 0.704545, train loss: 0.015162
batch:  20/24 , train acc: 0.704762, train loss: 0.030034
epoch:   3, train acc: 0.688935, train loss: 0.034798, test acc: 0.466667, test loss: 0.062017
Saved a model checkpoint at ./model_0104_2/checkpoints/sim_race_keyboad_test_ResNet18_epoch=3.pth

batch:  10/24 , train acc: 0.65    , train loss: 0.016171
batch:  20/24 , train acc: 0.67381 , train loss: 0.029299
epoch:   4, train acc: 0.672234, train loss: 0.033255
batch:  10/24 , train acc: 0

## train.pyの実行(上記でエラーが発生する場合)
train.pyを実行すれば動く。迷ったら以下のコードでtrain.pyを直接実行する。

Train.pyの修正箇所：
- CSVファイルにtrain_dataのパスがべた書きされているので、変更する必要あり。
-train.py:76行目
```
model_ckpt_path = args.model_ckpt_path_temp.format(args.dataset_name, args.model_name, epoch+1)
```
を以下に変更。モデル保存パスを引数から反映させるようにする。
```
model_ckpt_path_temp = args.model_ckpt_dir + '{}_{}_epoch={}.pth'
model_ckpt_path = model_ckpt_path_temp.format(args.dataset_name, args.model_name, epoch+1)
```

In [ ]:
# !time python train.py --n_epoch 20 --data_csv "../../../../DataSet/_2021-01-02-11-37-17/_2021-01-02-11-37-17.csv" --model_name sample_model --model_ckpt_dir "./model_0104/checkpoints/"

## 学習モデルの軽量化

環境うまく動かず失敗。jetson nanoのローカル環境にコピーしてから実行している。